**Import the necessary libraries**

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix,classification_report,plot_confusion_matrix
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import auc, plot_precision_recall_curve
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

**Load data into pandas dataframe**

In [2]:
recom_df = pd.read_csv(r'D:\MACHINE LEARNING\Mentorship program\ML Project2\recom.csv',index_col=0)

In [3]:
recom_df.head()

,Main_ID,Transaction_ID,Date,Price,Code_Product,Amount,ItemKey
0,90fada91,264f7a69,2022-10-07 20:53:49.153,125.0,5002.0,1.0,5002.0
1,9006f9ac,45c7d853,2022-09-17 15:54:57.187,19.0,35012.0,1.0,NaN
2,32270891,61ad76dd,2022-11-28 13:51:55.667,141.0,5005.0,1.0,5005.0
3,97e03e47,41ee09f6,2022-09-12 16:20:22.110,4.5,35078.5,1.0,NaN
4,41949228,244fe6d8,2022-10-14 18:53:43.933,129.5,49291.5,5.0,NaN


In [4]:
# To check the dataset for each user(Main_ID) separately
recom_df.sort_values(by='Main_ID')

,Main_ID,Transaction_ID,Date,Price,Code_Product,Amount,ItemKey
31628,00024de6,9016611c,2022-09-03 15:12:52.607,6.0,35097.0,2.0,NaN
37021,00084856,3a8c5a03,2022-10-21 16:40:07.373,24.0,49292.0,1.0,NaN
37004,00084856,cb1b492d,2022-11-05 15:08:11.743,22.0,49291.5,1.0,NaN
18346,0008e848,382c1a6b,2022-09-13 19:34:10.853,13.0,40002.0,1.0,NaN
3927,00096930,3bc7a8f5,2022-11-25 20:13:57.747,48.0,49291.5,2.0,NaN
...,...,...,...,...,...,...,...
39799,fff905d0,09b9a492,2022-10-17 13:45:20.843,24.0,49292.0,1.0,NaN
13259,fff9726b,de2af9b8,2022-11-10 11:35:57.950,66.0,45001.5,1.0,45001.5
38966,fff9726b,4ca53dbb,2022-09-23 15:50:50.903,9.0,30003.0,1.0,30003.0
35259,fffa332b,0ff3f909,2022-09-25 11:31:19.700,58.0,49556.0,1.0,NaN


In [5]:
recom_df.duplicated().sum()

171

In [6]:
recom_df['Transaction_ID'].value_counts()

9e0891a8    3
e2fdce56    3
78b64973    3
4fd75519    3
e71b5fae    3
           ..
96a11862    1
adc7d3dc    1
9a510f4e    1
0f1112fd    1
0ff8b41f    1
Name: Transaction_ID, Length: 48403, dtype: int64

In [7]:
recom_df[recom_df['Transaction_ID']=='e2fdce56']

,Main_ID,Transaction_ID,Date,Price,Code_Product,Amount,ItemKey
14083,4e1ade63,e2fdce56,2022-10-28 11:58:18.400,531.5,5009.0,1.0,5009.0
14586,4e1ade63,e2fdce56,2022-10-28 11:58:18.400,531.5,5009.0,1.0,5009.0
26563,4e1ade63,e2fdce56,2022-10-28 11:58:18.400,531.5,10023.0,1.0,10023.0


In [8]:
recom_df.drop_duplicates(inplace=True)

In [9]:
recom_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49829 entries, 0 to 49999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Main_ID         49829 non-null  object 
 1   Transaction_ID  49829 non-null  object 
 2   Date            49829 non-null  object 
 3   Price           49829 non-null  float64
 4   Code_Product    49829 non-null  float64
 5   Amount          49829 non-null  float64
 6   ItemKey         28512 non-null  float64
dtypes: float64(4), object(3)
memory usage: 3.0+ MB


In [10]:
# Since ItemKey and Code_Product columns are same with some of the values are missing in ItemKey. 
# Hence we kept the Code+Product and dropped the ItemKey.
# recom_df['ItemKey'].unique()

In [11]:
# recom_df['ItemKey1']=recom_df['ItemKey'].fillna(recom_df['Code_Product'])

In [12]:
# recom_df['ItemKeydiff']=recom_df['ItemKey1']-recom_df['Code_Product']

In [13]:
# recom_df['ItemKeydiff'].unique()

In [14]:
recom_df.drop('ItemKey',axis=1,inplace=True)

In [15]:
recom_df

,Main_ID,Transaction_ID,Date,Price,Code_Product,Amount
0,90fada91,264f7a69,2022-10-07 20:53:49.153,125.0,5002.0,1.0
1,9006f9ac,45c7d853,2022-09-17 15:54:57.187,19.0,35012.0,1.0
2,32270891,61ad76dd,2022-11-28 13:51:55.667,141.0,5005.0,1.0
3,97e03e47,41ee09f6,2022-09-12 16:20:22.110,4.5,35078.5,1.0
4,41949228,244fe6d8,2022-10-14 18:53:43.933,129.5,49291.5,5.0
...,...,...,...,...,...,...
49995,bb127ffb,4e0eb5ab,2022-09-24 21:48:20.847,111.5,45004.0,4.0
49996,a8bc484a,c9946c16,2022-11-18 19:49:01.973,34.0,49292.0,1.0
49997,c983862a,d1a35c5c,2022-11-24 20:02:43.023,178.0,5001.5,1.0
49998,8821da12,66f9b474,2022-11-06 13:07:01.423,26.0,49291.5,1.0


In [16]:
recom_df.sort_values(by='Main_ID',inplace=True)

In [17]:
recom_df.reset_index(drop=True)

,Main_ID,Transaction_ID,Date,Price,Code_Product,Amount
0,00024de6,9016611c,2022-09-03 15:12:52.607,6.0,35097.0,2.0
1,00084856,3a8c5a03,2022-10-21 16:40:07.373,24.0,49292.0,1.0
2,00084856,cb1b492d,2022-11-05 15:08:11.743,22.0,49291.5,1.0
3,0008e848,382c1a6b,2022-09-13 19:34:10.853,13.0,40002.0,1.0
4,00096930,3bc7a8f5,2022-11-25 20:13:57.747,48.0,49291.5,2.0
...,...,...,...,...,...,...
49824,fff905d0,09b9a492,2022-10-17 13:45:20.843,24.0,49292.0,1.0
49825,fff9726b,de2af9b8,2022-11-10 11:35:57.950,66.0,45001.5,1.0
49826,fff9726b,4ca53dbb,2022-09-23 15:50:50.903,9.0,30003.0,1.0
49827,fffa332b,0ff3f909,2022-09-25 11:31:19.700,58.0,49556.0,1.0


In [18]:
# recom_df[recom_df['Code_Product']==5002.5]

In [19]:
# recom_df.set_index('Main_ID')

In [20]:
d= pd.pivot_table(index='Main_ID',columns='Code_Product',values='Amount', data=recom_df) 
d

Code_Product,5000.5,5001.0,5001.5,5002.0,5002.5,5003.0,5004.0,5004.5,5005.0,5007.0,...,200035.5,200037.0,200037.5,200038.0,200038.5,200045.5,200046.0,200046.5,200047.0,350027.5
Main_ID,,,,,,,,,,,,,,,,,,,,,
00024de6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00084856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0008e848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00096930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000c66b7,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff8b1c4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fff905d0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fff9726b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Calculating the pearson correlation**

In [21]:
d.corr(method='pearson')

Code_Product,5000.5,5001.0,5001.5,5002.0,5002.5,5003.0,5004.0,5004.5,5005.0,5007.0,...,200035.5,200037.0,200037.5,200038.0,200038.5,200045.5,200046.0,200046.5,200047.0,350027.5
Code_Product,,,,,,,,,,,,,,,,,,,,,
5000.5,1.000000,0.041565,-0.086369,0.259041,NaN,NaN,NaN,-0.161382,-0.076829,-0.292770,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5001.0,0.041565,1.000000,0.124233,-0.111430,NaN,NaN,NaN,-0.301398,0.569109,-0.333333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5001.5,-0.086369,0.124233,1.000000,-0.097222,NaN,NaN,NaN,NaN,NaN,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5002.0,0.259041,-0.111430,-0.097222,1.000000,NaN,NaN,NaN,0.496139,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5002.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200045.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
200046.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200046.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
#A list of 20 products to recommend to a user who has purchased'5002.0'
# d.corr(method='pearson')[5002.0].sort_values(ascending=False).iloc[:20]

**We have to recommend multiple items for each user**<br>A list is created containing users and the items purchased by the user




In [23]:
# lst1 = list(zip(recom_df['Main_ID'], recom_df['Code_Product']))
lst=list(set(list(zip(recom_df['Main_ID'], recom_df['Code_Product']))))
# lst

**Converting the list into dictionary containing multiple items purchased by each user**

In [24]:
Code_Product_dict = dict()

for line in lst:
    if line[0] in Code_Product_dict:
        # append the new number to the existing array at this slot
        Code_Product_dict[line[0]].append(line[1])
    else:
        # create a new array in this slot
        Code_Product_dict[line[0]] = [line[1]]

In [25]:
Code_Product_dict

{'0a85aee3': [10004.0, 45002.5],
 'f54007bd': [48504.5, 5017.0, 45003.0],
 '57bdb962': [10013.0, 5000.5, 49291.5, 48513.0],
 '04ae83eb': [30002.0],
 'd951e9f5': [49292.0, 10024.0, 10004.0, 45004.0, 5012.0, 10015.0],
 '7238abbc': [49356.0],
 'abf696e1': [49291.5, 48513.0],
 '0ca98cbe': [15005.0],
 'd4918977': [49567.5,
  5017.0,
  49291.5,
  5025.0,
  5001.0,
  49556.0,
  5002.0,
  10003.0,
  48589.0,
  10032.5,
  48504.5,
  49427.5,
  25003.0,
  10004.0,
  45004.0,
  48513.0,
  10013.0,
  5000.5],
 '4aadbea7': [25003.5, 5000.5, 10001.0],
 '5fb1c30d': [5009.0],
 '60034029': [49567.5],
 '87fa22a4': [5009.0, 45001.0],
 '931b6ca7': [49291.5, 20034.5, 5002.0, 5011.5, 49567.5],
 '825907c3': [35076.0, 35077.0],
 '0f20f352': [5017.0, 5000.5],
 '02fe2cdb': [5000.5],
 '97d152e6': [30002.0],
 'f921fd4c': [30009.5, 15016.0],
 '05622cbd': [5001.5, 49292.0, 10004.0, 45001.0],
 '35095245': [5002.0, 25003.0],
 'cc2e71e6': [30008.0, 10022.5, 49129.5, 49129.0, 35000.5],
 'c893066c': [49292.0],
 '5035d7b

In [26]:
# len(Code_Product_dict)

In [27]:
# list(Code_Product_dict.keys())[0]

In [28]:
# def product_recommend(name):
#     if name in Code_Product_dict.keys():
#         index = list(Code_Product_dict).index(name)
#         print(index)
#         for j in range(len(list(Code_Product_dict.values())[index])):
#             dfr=pd.DataFrame(d.corr(method='pearson')[list(Code_Product_dict.values())[index][j]].sort_values(ascending=False).iloc[:11])
#             dfr.reset_index(inplace=True)
#             print('\n')
#             print(list(Code_Product_dict.keys())[index])
#             dfr_filtered = dfr[dfr['Code_Product'] != list(Code_Product_dict.values())[index][j]]
    
#             print(dfr_filtered)
#     else:
#         print('user does not exist')

Function created to pass Main_Id as a parameter to obtain recommendation for given Main_Id(User)

In [33]:
def product_recommend(name):
    code_pcoeff=[]
    
    if name in Code_Product_dict.keys():
        index = list(Code_Product_dict).index(name)
        print(index)
        for j in range(len(list(Code_Product_dict.values())[index])):
            dfr=pd.DataFrame(d.corr(method='pearson').fillna(0)[list(Code_Product_dict.values())[index][j]].round(3).sort_values(ascending=False).iloc[:5])
            dfr.reset_index(inplace=True)
#             print('\n')
#             print(list(Code_Product_dict.keys())[index])
            dfr_filtered = dfr[dfr['Code_Product'] != list(Code_Product_dict.values())[index][j]]
            dfr_filtered.reset_index(drop=True, inplace=True)
            for i in range(2):
                cp=tuple(list(dfr_filtered.iloc[i]))
                code_pcoeff.append(cp)
    else:
        return 'user does not exist'
        
        
    if len(code_pcoeff)>10:
        code_pcoeff=code_pcoeff[slice(10)]
    else:
        code_pcoeff=code_pcoeff 
        
           
    first_data_code_pcoeff = [t[0] for t in code_pcoeff]
    dict1={'List of items purchased by user':list(Code_Product_dict.values())[index]}
    dict2={'List of items recommended to user':first_data_code_pcoeff}
    return dict1,dict2
#     return 'List of items purchased by user',list(Code_Product_dict.values())[index],'List of items recommended to user', first_data_code_pcoeff

In [35]:
product_recommend('d4918977')

8


({'List of items purchased by user': [49567.5,
   5017.0,
   49291.5,
   5025.0,
   5001.0,
   49556.0,
   5002.0,
   10003.0,
   48589.0,
   10032.5,
   48504.5,
   49427.5,
   25003.0,
   10004.0,
   45004.0,
   48513.0,
   10013.0,
   5000.5]},
 {'List of items recommended to user': [10015.5,
   10033.0,
   15004.5,
   5007.0,
   35071.5,
   35006.5,
   15000.5,
   10004.0,
   35097.0,
   40075.5]})